<a href="https://colab.research.google.com/github/ganbozza/test/blob/main/Mega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @markdown <center><h2>Install MEGAcmd</h2></center><br>
# @markdown <center><h4>first step is to run this cell</h4></center><br>
from functools import wraps
import errno
import os
import signal
import subprocess
import shlex

if not os.path.exists("/root/.ipython/ocr.py"):
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh

class TimeoutError(Exception):
    pass


def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator



@timeout(10)
def runShT(args):
    return runSh(args, output=True)


# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libc-ares2 libmediainfo0v5 libzen0v5')
    runSh('sudo curl -sL -o /var/cache/apt/archives/libssl1.1.deb http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/libssl1.1.deb', output=True)
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/xUbuntu_21.10/amd64/megacmd-xUbuntu_21.10_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    if os.path.exists("/usr/bin/mega-cmd"):
      print("MEGAcmd is installed.")
    else:
        raise Exception("MEGAcmd is not installed")

In [ ]:
!rclone config